In [3]:
%matplotlib inline

In [35]:
from PIL import Image, ImageDraw
import math
import random

def build_data(a, n=4, R=20):
    img = Image.new(mode='RGB', size=(R*3, R*3), color='white')
    draw = ImageDraw.Draw(img)
    pts = []
    for i in range(1, 5):
        
        A = a*180/math.pi
        y = math.cos(A)*R+R
        x = math.sin(A)*R+R
        pts.append((i, x, y))
        a += 10.0

    random.shuffle(pts)
    xys = []
    ids = []
    for i, (id, x, y) in enumerate(pts):
        ids.append(id)
        xys.append(x)
        xys.append(y)
    return xys, ids
#         draw.text((x, y), str(i+1), fill='black')
#     return img, ids

# img, ids = build_data(45)

# import matplotlib.pyplot as plt
# plt.figure()
# plt.imshow(img)
# plt.show()


In [36]:

IDS_TABLE = []
for i1 in [1,2,3,4]:
    ids = [i1]
    for i2 in [1,2,3,4]:
        if i2 in ids:
            continue
        ids.append(i2)
        for i3 in [1,2,3,4]:
            if i3 in ids:
                continue
            ids.append(i3)
            for i4 in [1,2,3,4]:
                if i4 in ids:
                    continue
                ids.append(i4)
                IDS_TABLE.append(''.join([str(id) for id in ids]))
                del ids[-1]
            del ids[-1]
        del ids[-1]
    
                                 
print(IDS_TABLE)

def encode_ids(ids):
    return IDS_TABLE.index(''.join([str(id) for id in ids]))

def decode_ids(index):
    return IDS_TABLE[index]

print(encode_ids([1,3,4,2]))

['1234', '1243', '1324', '1342', '1423', '1432', '2134', '2143', '2314', '2341', '2413', '2431', '3124', '3142', '3214', '3241', '3412', '3421', '4123', '4132', '4213', '4231', '4312', '4321']
3


In [37]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485], [0.229])
    ])

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(IDS_TABLE))

    def forward(self, x):
        x = x.view(-1, 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



In [40]:
import numpy as np

model.train()
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(100):
        img, ids = build_data(random.randrange(.0, 80.0))
        inputs = np.array(img)
        inputs = torch.tensor(inputs).unsqueeze(0)
        print(inputs)
#         inputs = transform(inputs).unsqueeze(0)
#         print(inputs.shape)
        
        labels = encode_ids(ids)
        labels = torch.tensor([labels])
#         print(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
#         print(outputs, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
#         break

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

tensor([[1.2489e+01, 1.4638e+00, 1.2568e+01, 3.8568e+01, 3.4388e+01, 6.1084e+00,
         4.4379e-05, 2.0042e+01]], dtype=torch.float64)


RuntimeError: shape '[-1, 48]' is invalid for input of size 8